In [9]:
import coiled

cluster = coiled.Cluster(n_workers=2, backend_options={"region_name": "us-east-1"})
cluster.adapt(minimum=10, maximum=200)
client = cluster.get_client()

Output()

Output()

2023-08-18 14:55:25,648 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=10 maximum=200


In [10]:
client

<Client: 'tls://10.0.87.243:8786' processes=2 threads=8, memory=29.70 GiB>

## Setup

In [1]:
%load_ext watermark

import dask
import distributed
import flox  # make sure its available
import fsspec
import numpy as np
import xarray as xr
import zarr

xr.set_options(
    display_expand_attrs=False,
    display_expand_coords=False,
    display_expand_data=True,
)

%watermark -iv

zarr       : 2.15.0
fsspec     : 2023.6.0
flox       : 0.7.2
numpy      : 1.25.2
xarray     : 2023.6.0
distributed: 2023.8.0
dask       : 2023.8.0



## Load NWM data

In [6]:
ds = xr.open_dataset(
    fsspec.get_mapper(
        "s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr", anon=True
    ),
    consolidated=True,
    inline_array=True,
    engine="zarr",
    chunks={},
)
ds

<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates: (7)
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes: (5)

In [7]:
resampler = ds.resample(time="A-SEP", label="right", skipna=True)

## Rechunk to begin new chunks at group boundaries

## Resample

In [8]:
%%time

# Get the maximum value for each data variable in the dataset
ds_WY_max = resampler.max()
ds_WY_max = ds_WY_max.rename(
    {"streamflow": "streamflow_max", "velocity": "velocity_max"}
)

# Get the minimum value for each data variable in the dataset
ds_WY_min = resampler.min()
ds_WY_min = ds_WY_min.rename(
    {"streamflow": "streamflow_min", "velocity": "velocity_min"}
)

ds_out = xr.merge([ds_WY_max, ds_WY_min], compat="override")
ds_out

CPU times: user 4.38 s, sys: 205 ms, total: 4.58 s
Wall time: 5.86 s


<xarray.Dataset>
Dimensions:         (time: 43, feature_id: 2776738)
Coordinates: (7)
Data variables:
    streamflow_max  (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    velocity_max    (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    streamflow_min  (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    velocity_min    (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
Attributes: (5)

In [ ]:
ds_out.to_zarr("s3://cherian-scratch/nwm-streamflow-water-year.zarr", mode="w")